# Call Context Summary

## 1. Setup Workspace

In [1]:
import json
import pathlib
import copy
import operator

import anytree
import networkx as nx
import sympy

import paptree

## 2. Load Data

The fibonnaci library contains fours implementations of a fibonacci generator:

1. Recursive (naive)
2. Recursive (memoized)
3. Iterative
4. Lookup table

The library unit tests verify the first eight fibonnaci numbers returned by each implementation as well as the behavior when the user requests a fibonnaci number to large to fit in the return type. This should result in 36 (`4 * (8 + 1)`) traces.

In [2]:
trace_file = pathlib.Path.cwd().parent / "data/fibonacci/paptrace.json"
trees = paptree.utils.from_file(trace_file)
print(f"Loaded {len(trees)} traces.")

Loaded 36 traces.


## 3. Node Summary

### 3.1 Unfiltered Nodes

In [3]:
nodes = []
for tree in trees:
    nodes.extend(anytree.PreOrderIter(tree.root))
print(f"Node count: {len(nodes)}")

Node count: 4974


In [4]:
node_types = {}
for node in nodes:
    node_types.setdefault(node.type, []).append(node)
print(f"Node type count: {len(node_types)}")

Node type count: 7


In [5]:
print("Nodes per type:")
for k, v in node_types.items():
    print(f"- {k}: {len(v)}")

Nodes per type:
- CalleeExpr: 1354
- IfThenStmt: 923
- ReturnStmt: 1350
- CallerExpr: 1316
- CXXThrowExpr: 4
- ForStmt: 6
- LoopIter: 21


### 3.2. Unique Nodes

In [6]:
unique_nodes = []
for node in nodes:
    if node not in unique_nodes:
        unique_nodes.append(node)
print(f"Unique node count: {len(unique_nodes)}")

Unique node count: 326


In [7]:
# We expect this count to match the unfiltered node type count.
unique_node_types = {}
for node in unique_nodes:
    unique_node_types.setdefault(node.type, []).append(node)
print(f"Node type count: {len(unique_node_types)}")

Node type count: 7


In [8]:
print("Nodes per type:")
for k, v in unique_node_types.items():
    print(f"- {k}: {len(v)}")

Nodes per type:
- CalleeExpr: 101
- IfThenStmt: 7
- ReturnStmt: 7
- CallerExpr: 205
- CXXThrowExpr: 4
- ForStmt: 1
- LoopIter: 1


We can reason about the control flow related unique node counts using the source:

- The source has 7 `if` statements (`grep if src/fibonacci.cpp | wc -l`).
    - Note: This includes `else if`.
- The source has 7 `return` statements (`grep return src/fibonacci.cpp | wc -l`).
- The source has 4 `throw` statements (`grep throw src/fibonacci.cpp | wc -l`).
- The source has 1 `for` statement (`grep "for " src/fibonacci.cpp | wc -l`).

## 4. Trace Summary

As mentioned in Section 2, we expect 9 traces for each of the 4 exposed library functions for a total of 36 traces.

In [9]:
binned_trees = {}
for tree in trees:
    binned_trees.setdefault(tree.root.sig, []).append(tree)
print(f"Trace entry point count: {len(binned_trees)}")

Trace entry point count: 4


In [10]:
print("Traces per entry point:")
for k, v in binned_trees.items():
    print(f"- {k}: {len(v)}")

Traces per entry point:
- unsigned long long fibonacci::RecursiveNaive(unsigned short): 9
- unsigned long long fibonacci::RecursiveMemo(unsigned short): 9
- unsigned long long fibonacci::Iterative(unsigned short): 9
- unsigned long long fibonacci::LookupTable(unsigned short): 9


## 5. Trace Analysis

### 5.1. Recursive (Naive)

In [11]:
rec_naive_sig = f"unsigned long long fibonacci::RecursiveNaive(unsigned short)"
rec_naive_trees = binned_trees[rec_naive_sig]

def to_params_str(params):
    #return ", ".join([f"{param['name']}={param['value']}" for param in params])
    return ", ".join([f"{param['value']}" for param in params])

def to_call_str(node):
    return f"{node.sig}: ({to_params_str(node.params)})"

print("Recorded calls:")
for tree in rec_naive_trees:
    print(f"{to_call_str(tree.root)}")

Recorded calls:
unsigned long long fibonacci::RecursiveNaive(unsigned short): (0)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (1)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (2)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (3)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (4)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (5)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (6)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (7)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (94)


In [12]:
# Peek at select traces that we know take differing paths.
def to_simple_node_view(node):
    sym = "sym @ " if hasattr(node, "target") else ""
    if paptree.Node.is_call_type(node.type):
        desc = f" {to_call_str(node)}"
    else:
        desc = f" {node.type}: {node.desc}"
    return f"({sym}{node.name}){desc}"

for tree in operator.itemgetter(0, 2, 8)(rec_naive_trees):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (0)
└── (2106009) IfThenStmt: n < 2
    └── (2106007) ReturnStmt: return n

(2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (2)
└── (2106188) ReturnStmt: return RecursiveNaive(n - 1) + RecursiveNaive(n - 2)
    ├── (2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (1)
    │   └── (2106009) IfThenStmt: n < 2
    │       └── (2106007) ReturnStmt: return n
    ├── (2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (0)
    │   └── (2106009) IfThenStmt: n < 2
    │       └── (2106007) ReturnStmt: return n
    └── (2106184) unsigned long long operator+(unsigned long long, unsigned long long): (1, 0)
        └── (2106117) int operator-(unsigned short, int): (2, 1)
            ├── (2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (1)
            │   └── (2106009) IfThenStmt: n < 2
            │       └── (2106007) ReturnStmt: return n


In [13]:
def is_cf_node(node):
    return node.type in ["IfThenStmt", "ReturnStmt", "CXXThrowExpr", "ForStmt", "LoopIter"]

# NOTE: We currently do not need to account for the context of calls made inside the trace
# because we have not "squashed" those call nodes. If we did "squash" call nodes, then we
# would need to query the path for that call to be able to correctly form partitions.
fn_paths = {}
for tree in operator.itemgetter(0, 2, 3, 8)(rec_naive_trees):
    #print(f"\n{fn_name}({to_params_str(tree.root.params)})")
    cf_nodes = [node.name for node in anytree.PreOrderIter(tree.root, filter_=is_cf_node)]
    fn_paths.setdefault(tuple(cf_nodes), []).append(tree)
    #print(anytree.RenderTree(tree), "\n")

# Q. What do we expect here?
#
# A1. We expect 3 paths. Each trace in n > 1 =< 93 takes Path 2. Maximum of 3 paths.
# Path 1: n <= 1
# Path 2: n > 1 =< 93
# Path 3: n > 93
#
# A2. We expect 4 paths. Each trace in n > 1 =< 93 takes its own Path. Maximum of 94 paths.
# Path 1: n <= 1
# Path 2: n == 2
# Path 3: n > 93
# 
# Clearly A1 is preferable to A2, but our current code produces A2.

print(f"Path count: {len(fn_paths)}", "\n")
print("Traces per path:")
for k, v in fn_paths.items():
    print(f"- {k}: {len(v)}")

Path count: 4 

Traces per path:
- (2106009, 2106007): 1
- (2106188, 2106009, 2106007, 2106009, 2106007, 2106009, 2106007, 2106009, 2106007): 1
- (2106188, 2106188, 2106009, 2106007, 2106009, 2106007, 2106009, 2106007, 2106009, 2106007, 2106009, 2106007, 2106188, 2106009, 2106007, 2106009, 2106007, 2106009, 2106007, 2106009, 2106007, 2106009, 2106007): 1
- (2106081, 2106075): 1


In [14]:
# Getting to A2.
#
# 1. First thing we can do is "squash" calls so that recursion does not result in differing paths.
# 2. Next, we can use the expression for the "squashed" calls to differentiate between calls to different paths.
# 2a. For now, we can just fake an expression for the "squashed" calls.
# 2b. In practice, we will need to solve for the squashed calls using symbolic regression.

# STEP 1
# Let's start by putting our trace root nodes in a container that allows us
# to lookup the trace by context.
# NOTE: This does not account for complete non-root traces (e.g., RecursiveMemoImpl).
trace_roots = {}
for tree in trees:
    call_str = to_call_str(tree.root)
    if call_str not in trace_roots:
        trace_roots[call_str] = tree.root
    else:
        if tree.root != trace_roots[call_str]:
            raise RuntimeError(
                "Unhandled scenario: 2 traces with the same context have differing trees.")
            
# Now we walk the trees looking for child nodes that can be substituted with a
# trace root node.
for tree in trees:
    for node in anytree.PreOrderIter(tree.root):
        if node.is_root:
            continue
        replace_children = False
        new_children = []
        for child in node.children:
            if child.type == "CalleeExpr":
                child_call_str = to_call_str(child)
                if child_call_str in trace_roots:
                    replace_children = True
                    child = anytree.SymlinkNode(child)
            new_children.append(child)
        if replace_children:
            node.children = tuple(new_children)
            
# Peek again at the select traces now that calls have been "squashed".
for tree in operator.itemgetter(0, 2, 8)(rec_naive_trees):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (0)
└── (2106009) IfThenStmt: n < 2
    └── (2106007) ReturnStmt: return n

(2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (2)
└── (2106188) ReturnStmt: return RecursiveNaive(n - 1) + RecursiveNaive(n - 2)
    ├── (sym @ 2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (1)
    ├── (sym @ 2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (0)
    └── (2106184) unsigned long long operator+(unsigned long long, unsigned long long): (1, 0)
        └── (2106117) int operator-(unsigned short, int): (2, 1)
            ├── (sym @ 2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (1)
            └── (2106165) int operator-(unsigned short, int): (2, 2)
                └── (sym @ 2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (0)

(2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (94)
└── (21060

In [15]:
# Let's group by path again. We expect to have a path count of 3.
fn_paths = {}
for tree in rec_naive_trees:
    cf_nodes = [node.name for node in anytree.PreOrderIter(tree.root, filter_=is_cf_node)]
    fn_paths.setdefault(tuple(cf_nodes), []).append(tree)

print(f"Path count: {len(fn_paths)}", "\n")
print("Traces per path:")
for k, v in fn_paths.items():
    print(f"- {k}: {len(v)}")

Path count: 3 

Traces per path:
- (2106009, 2106007): 2
- (2106188,): 6
- (2106081, 2106075): 1


In [16]:
# STEP 2
#
# At this point we've only performed partial path partitioning. For example, RecursiveNaive(2)
# and RecursiveNaive(3) seem to take the same path, but we don't know until we've solved for
# the complexity of their recursive calls.
# 
# RN(2) depends on both RN(1) and RN(0)
# RN(3) depends on both RN(2) and RN(1)
#
# This means these relationships can be modeled by the dependency tree:
# RN(3)
# + RN(2)
#   + RN(1)
#   + RN(0)
#
# We need to generate this dependency tree using code.

G = nx.DiGraph()
G.add_node("root")
for tree in rec_naive_trees:
    nodes = [node for node in anytree.PreOrderIter(tree.root, filter_=lambda n: hasattr(n, "target"))]
    target_str = to_call_str(tree.root)
    if not nodes:
        G.add_edge(target_str, "root")
    else:
        for node in nodes:
            prereq_str = to_call_str(node)
            G.add_edge(prereq_str, target_str)

dfs_postorder = [node for node in nx.dfs_postorder_nodes(G)]
eval_order = dfs_postorder[::-1]

print("\nEval Order:")
for x in eval_order:
    print(x)


Eval Order:
unsigned long long fibonacci::RecursiveNaive(unsigned short): (94)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (1)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (0)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (2)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (3)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (4)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (5)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (6)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (7)
root


In [17]:
# Convert path tuples to IDs.
# We store an ID->trees mapping so that we can easily query trees for a path ID.
path_ids = {}
id_paths = {}
for path_tuple, trees in fn_paths.items():
    path_id = path_ids.setdefault(path_tuple, len(path_ids))
    id_paths[path_id] = trees

print(id_paths.keys())

dict_keys([0, 1, 2])


In [33]:
# Let's structure traces for the solver.
# We start with Path 0 (constant time).

# It is expected that we start with some known values.
# NOTE: This currently uses signatures to generalize (i.e., it is not accounting for context).
known = {
    "int operator-(unsigned short, int)": ["C_OP_1"], 
    "unsigned long long operator+(unsigned long long, unsigned long long)": ["C_OP_2"],
}
exprs = {}
call_path_ids = {}
sympy_exprs = {}

def to_expr(node, level=0):
    subexpr = []
    for child in node.children:
        subexpr.extend(to_expr(child, level+1))
    if node.is_root:
        subexpr.append(f"C_P{call_path_ids[to_call_str(node)]}")
    elif paptree.Node.is_call_type(node.type):
        #print(f"{' ' * 2 * level}including node {to_simple_node_view(node)}")
        if node.sig in known:
            subexpr.extend(known[node.sig])
        else:
            subexpr.extend(exprs[to_call_str(node)])
    else:
        #print(f"{' ' * 2 * level}excluding node {to_simple_node_view(node)}")
        pass
    return subexpr

for path_id, path_trees in id_paths.items():
    print(f"\nPATH {path_id}")
    path_exprs = {}
    for tree in path_trees:
        call_path_ids[to_call_str(tree.root)] = path_id
        expr = to_expr(tree.root)
        exprs[to_call_str(tree.root)] = expr
        path_exprs[to_call_str(tree.root)] = expr # This is just used for printing.
    for k, v in path_exprs.items():
        sympy_expr = sympy.sympify(' + '.join(v))
        sympy_exprs[to_call_str(tree.root)] = sympy_expr
        print(f"{k}: {sympy_expr}")


PATH 0
unsigned long long fibonacci::RecursiveNaive(unsigned short): (0): C_P0
unsigned long long fibonacci::RecursiveNaive(unsigned short): (1): C_P0

PATH 1
unsigned long long fibonacci::RecursiveNaive(unsigned short): (2): 2*C_OP_1 + C_OP_2 + 4*C_P0 + C_P1
unsigned long long fibonacci::RecursiveNaive(unsigned short): (3): 6*C_OP_1 + 3*C_OP_2 + 10*C_P0 + 3*C_P1
unsigned long long fibonacci::RecursiveNaive(unsigned short): (4): 18*C_OP_1 + 9*C_OP_2 + 28*C_P0 + 9*C_P1
unsigned long long fibonacci::RecursiveNaive(unsigned short): (5): 50*C_OP_1 + 25*C_OP_2 + 76*C_P0 + 25*C_P1
unsigned long long fibonacci::RecursiveNaive(unsigned short): (6): 138*C_OP_1 + 69*C_OP_2 + 208*C_P0 + 69*C_P1
unsigned long long fibonacci::RecursiveNaive(unsigned short): (7): 378*C_OP_1 + 189*C_OP_2 + 568*C_P0 + 189*C_P1

PATH 2
unsigned long long fibonacci::RecursiveNaive(unsigned short): (94): C_P2


### 5.2. Recursive (Memoized)

In [19]:
rec_memo_sig = f"unsigned long long fibonacci::RecursiveMemo(unsigned short)"
rec_memo_trees = binned_trees[rec_memo_sig]

print("Recorded calls:")
for tree in rec_memo_trees:
    print(f"{to_call_str(tree.root)}")

Recorded calls:
unsigned long long fibonacci::RecursiveMemo(unsigned short): (0)
unsigned long long fibonacci::RecursiveMemo(unsigned short): (1)
unsigned long long fibonacci::RecursiveMemo(unsigned short): (2)
unsigned long long fibonacci::RecursiveMemo(unsigned short): (3)
unsigned long long fibonacci::RecursiveMemo(unsigned short): (4)
unsigned long long fibonacci::RecursiveMemo(unsigned short): (5)
unsigned long long fibonacci::RecursiveMemo(unsigned short): (6)
unsigned long long fibonacci::RecursiveMemo(unsigned short): (7)
unsigned long long fibonacci::RecursiveMemo(unsigned short): (94)


In [20]:
# Peek at select traces that we know take differing paths.
for tree in operator.itemgetter(0, 2, 8)(rec_memo_trees):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(2109686) unsigned long long fibonacci::RecursiveMemo(unsigned short): (0)
├── (2106382) int operator+(unsigned short, int): (0, 1)
├── (2109647) value_type operator=(value_type, int): (0, 1)
│   └── (2109633) reference std::vector<unsigned long long>::operator[](size_type): ({ 0 }, 1)
└── (2109684) ReturnStmt: return RecursiveMemoImpl(n, memo)
    └── (2105928) unsigned long long fibonacci::(anonymous namespace)::RecursiveMemoImpl(unsigned short, std::vector<unsigned long long> &): (0, { 0 })
        └── (2105926) ReturnStmt: return memo[n]
            └── (2105916) reference std::vector<unsigned long long>::operator[](size_type): ({ 0 }, 0)

(2109686) unsigned long long fibonacci::RecursiveMemo(unsigned short): (2)
├── (2106382) int operator+(unsigned short, int): (2, 1)
├── (2109647) value_type operator=(value_type, int): (0, 1)
│   └── (2109633) reference std::vector<unsigned long long>::operator[](size_type): ({ 0, 0, 0 }, 1)
└── (2109684) ReturnStmt: return RecursiveMemoImpl(n, m

### 5.3. Iterative

In [21]:
iter_sig = f"unsigned long long fibonacci::Iterative(unsigned short)"
iter_trees = binned_trees[iter_sig]

print("Recorded calls:")
for tree in iter_trees:
    print(f"{to_call_str(tree.root)}")

Recorded calls:
unsigned long long fibonacci::Iterative(unsigned short): (0)
unsigned long long fibonacci::Iterative(unsigned short): (1)
unsigned long long fibonacci::Iterative(unsigned short): (2)
unsigned long long fibonacci::Iterative(unsigned short): (3)
unsigned long long fibonacci::Iterative(unsigned short): (4)
unsigned long long fibonacci::Iterative(unsigned short): (5)
unsigned long long fibonacci::Iterative(unsigned short): (6)
unsigned long long fibonacci::Iterative(unsigned short): (7)
unsigned long long fibonacci::Iterative(unsigned short): (94)


In [22]:
# Peek at select traces that we know take differing paths.
for tree in operator.itemgetter(0, 2, 8)(iter_trees):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(2110045) unsigned long long fibonacci::Iterative(unsigned short): (0)
└── (2109766) IfThenStmt: n < 2
    └── (2109764) ReturnStmt: return n

(2110045) unsigned long long fibonacci::Iterative(unsigned short): (2)
├── (2110029) ForStmt: for (unsigned short i = 2; i <= n; ++i)
│   └── (2110029) LoopIter: LoopIter
│       ├── (2109990) unsigned long long operator=(unsigned long long, unsigned long long): (105553123975920, 1)
│       │   └── (2109986) unsigned long long operator+(unsigned long long, unsigned long long): (0, 1)
│       ├── (2110005) unsigned long long operator=(unsigned long long, unsigned long long): (0, 1)
│       └── (2110020) unsigned long long operator=(unsigned long long, unsigned long long): (1, 1)
└── (2110043) ReturnStmt: return fib

(2110045) unsigned long long fibonacci::Iterative(unsigned short): (94)
└── (2109827) IfThenStmt: n > 93
    └── (2109821) CXXThrowExpr: throw std::overflow_error("n must be less than 94")



### 5.4. Lookup Table

In [23]:
lookup_sig = f"unsigned long long fibonacci::Iterative(unsigned short)"
lookup_trees = binned_trees[lookup_sig]

print("Recorded calls:")
for tree in lookup_trees:
    print(f"{to_call_str(tree.root)}")

Recorded calls:
unsigned long long fibonacci::Iterative(unsigned short): (0)
unsigned long long fibonacci::Iterative(unsigned short): (1)
unsigned long long fibonacci::Iterative(unsigned short): (2)
unsigned long long fibonacci::Iterative(unsigned short): (3)
unsigned long long fibonacci::Iterative(unsigned short): (4)
unsigned long long fibonacci::Iterative(unsigned short): (5)
unsigned long long fibonacci::Iterative(unsigned short): (6)
unsigned long long fibonacci::Iterative(unsigned short): (7)
unsigned long long fibonacci::Iterative(unsigned short): (94)


In [24]:
# Peek at select traces that we know take differing paths.
# Note: Unlike preview implementations, calls for number 0 and 2 take the same path.
for tree in operator.itemgetter(0, 2, 8)(lookup_trees):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(2110045) unsigned long long fibonacci::Iterative(unsigned short): (0)
└── (2109766) IfThenStmt: n < 2
    └── (2109764) ReturnStmt: return n

(2110045) unsigned long long fibonacci::Iterative(unsigned short): (2)
├── (2110029) ForStmt: for (unsigned short i = 2; i <= n; ++i)
│   └── (2110029) LoopIter: LoopIter
│       ├── (2109990) unsigned long long operator=(unsigned long long, unsigned long long): (105553123975920, 1)
│       │   └── (2109986) unsigned long long operator+(unsigned long long, unsigned long long): (0, 1)
│       ├── (2110005) unsigned long long operator=(unsigned long long, unsigned long long): (0, 1)
│       └── (2110020) unsigned long long operator=(unsigned long long, unsigned long long): (1, 1)
└── (2110043) ReturnStmt: return fib

(2110045) unsigned long long fibonacci::Iterative(unsigned short): (94)
└── (2109827) IfThenStmt: n > 93
    └── (2109821) CXXThrowExpr: throw std::overflow_error("n must be less than 94")



# OLD CODE BELOW

## 3. Extract Call Nodes

In [25]:
call_nodes = []
for tree in trees:
    call_nodes.extend(
        anytree.findall(tree.root, filter_=lambda n: isinstance(n, paptree.CallNode)))

print(f"Number of call nodes: {len(call_nodes)}")

Number of call nodes: 1


In [26]:
print(f"Unique call nodes: {len(set([str(n) for n in call_nodes]))}")

Unique call nodes: 1


In [27]:
sigs = set([node.sig for node in call_nodes])
print(f"Number of unique signatures: {len(sigs)}")
display(sigs)

Number of unique signatures: 1


{'unsigned long long fibonacci::RecursiveNaive(unsigned short)'}

### 3.1. Caller Nodes

Caller nodes represent a call to an uninstrumented function. The complexity of these nodes must be provided by the user since there is no trace data to analyze.

In [28]:
caller_nodes = [node for node in call_nodes if node.type == "CallerExpr"]
print(f"Number of caller nodes: {len(caller_nodes)}")

caller_node_sigs = set([node.sig for node in caller_nodes])
print(f"Number of unique caller sigs: {len(caller_node_sigs)}")
print("Unique caller node sigs:")
print("\n".join(caller_node_sigs))

Number of caller nodes: 0
Number of unique caller sigs: 0
Unique caller node sigs:



### 3.2. Callee Nodes

Callee nodes represent a call to an instrumented function. The complexity of these nodes can be deduced from the collected trace data, assuming that a user has provided the complexity for any dependent caller nodes.

In [29]:
callee_nodes = [node for node in call_nodes if node.type == "CalleeExpr"]
print(f"Number of callee nodes: {len(callee_nodes)}")

callee_node_sigs = set([node.sig for node in callee_nodes])
print(f"Number of unique callee sigs: {len(callee_node_sigs)}")
print("Unique callee node sigs:")
print("\n".join(callee_node_sigs))

Number of callee nodes: 1
Number of unique callee sigs: 1
Unique callee node sigs:
unsigned long long fibonacci::RecursiveNaive(unsigned short)


In [30]:
# Convert operator nodes to CallNodes.
for tree in trees:
    for node in anytree.PreOrderIter(tree.root):
        if node.type in ["operator=", "operator+", "operator-", "operator++"]:
            
            print(node)


for tree in trees[18:22]:
    print(anytree.RenderTree(tree), "\n")